In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, concatenate
import keras_tuner as kt

# Build model function for hyperparameter tuning
def build_model(hp):
    units_1 = hp.Int('units_1', 32, 128, 32)
    units_2 = hp.Int('units_2', 32, 128, 32)
    learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

    input_1 = Input(shape=(10,), name="input_1")
    x1 = Dense(units_1, activation='relu')(input_1)
    x1 = Dense(units_2, activation='relu')(x1)

    input_2 = Input(shape=(64, 64, 3), name="input_2")
    x2 = Conv2D(32, (3, 3), activation='relu')(input_2)
    x2 = Conv2D(64, (3, 3), activation='relu')(x2)
    x2 = Flatten()(x2)

    combined = concatenate([x1, x2])
    output_1 = Dense(1, name="output_1")(combined)
    output_2 = Dense(10, activation='softmax', name="output_2")(combined)

    model = Model(inputs=[input_1, input_2], outputs=[output_1, output_2])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss={'output_1': 'mse', 'output_2': 'categorical_crossentropy'},
        metrics={'output_1': 'mae', 'output_2': 'accuracy'}
    )
    return model

# Hyperparameter tuning setup
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='multi_input_output_tuning'
)

# Generate data
x1_data, x2_data = np.random.rand(1000, 10), np.random.rand(1000, 64, 64, 3)
y1_data, y2_data = np.random.rand(1000, 1), tf.keras.utils.to_categorical(np.random.randint(0, 10, (1000, 1)), 10)
x1_train, x1_val = x1_data[:800], x1_data[800:]
x2_train, x2_val = x2_data[:800], x2_data[800:]
y1_train, y1_val = y1_data[:800], y1_data[800:]
y2_train, y2_val = y2_data[:800], y2_data[800:]

# Tuning
tuner.search([x1_train, x2_train], [y1_train, y2_train], validation_data=([x1_val, x2_val], [y1_val, y2_val]), epochs=5, batch_size=32)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

# Train best model
best_model.fit([x1_train, x2_train], [y1_train, y2_train], epochs=5, batch_size=32)


Trial 10 Complete [00h 00m 33s]
val_loss: 2.4155434370040894

Best val_loss So Far: 2.3904792070388794
Total elapsed time: 00h 04m 56s


/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_2             │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_1             │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 62, 62,    │        896 │ input_2[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 96)        │      1,056 │ input_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 60, 60,    │     18,496 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 96)        │      9,312 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 230400)    │          0 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 230496)    │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_1 (Dense)    │ (None, 1)         │    230,497 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_2 (Dense)    │ (None, 10)        │  2,304,970 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,565,227 (9.79 MB)

 Trainable params: 2,565,227 (9.79 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - loss: 2.3899 - output_1_loss: 0.0877 - output_1_mae: 0.2464 - output_2_accuracy: 0.1229 - output_2_loss: 2.3023
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 2.3847 - output_1_loss: 0.0827 - output_1_mae: 0.2431 - output_2_accuracy: 0.0877 - output_2_loss: 2.3020
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - loss: 2.3763 - output_1_loss: 0.0779 - output_1_mae: 0.2389 - output_2_accuracy: 0.1450 - output_2_loss: 2.2984
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - loss: 2.3790 - output_1_loss: 0.0800 - output_1_mae: 0.2448 - output_2_accuracy: 0.1066 - output_2_loss: 2.2990
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - loss: 2.3834 - output_1_loss: 0.0815 - output_1_mae: 0.2437 - output_2_accuracy: 0.1001 - output_2_loss: 2.3019
